### Define helper functionality to visualize forward and inverse image mapping

In [ ]:
from dataclasses import dataclass, field
from pathlib import Path
from typing import Callable, Optional

import numpy as np
import tqdm

from mynd.backend import metashape as backend

from mynd.camera import CameraID, CameraCalibration
from mynd.collections import CameraGroup, StereoCameraGroup
from mynd.image import Image, ImageLoader, filter_image_clahe

from mynd.geometry import HitnetModel, load_hitnet
from mynd.geometry import StereoGeometry, compute_stereo_geometry
from mynd.geometry import (
    StereoRectificationResult,
    compute_stereo_rectification,
)

from mynd.visualization import (
    WindowHandle,
    create_window,
    render_image,
    destroy_all_windows,
    wait_key_input,
    colorize_values,
)

from mynd.visualization import (
    StereoWindows,
    create_stereo_windows,
    render_stereo_geometry,
)

from mynd.utils.containers import Pair
from mynd.utils.environment import Environment, load_environment
from mynd.utils.key_codes import KeyCode
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


GroupID = CameraGroup.Identifier


def create_clahe_filter(clip: float, size: int) -> Callable:
    """Creates a CLAHE filter with the given parameters."""

    def clahe_filter(image: Image) -> Image:
        return filter_image_clahe(image, clip=clip, size=size)

    return clahe_filter


def filter_disparity_map(disparity: np.ndarray) -> np.ndarray:
    """Filter a disparity map."""
    # TODO: Add disparity filter
    return disparity


@dataclass
class StereoGeometryBatch:
    """Class representing batch for stereo geometry estimation."""

    # Data members
    calibrations: Pair[CameraCalibration]
    camera_pairs: list[Pair[CameraID]]
    image_loaders: dict[CameraID, ImageLoader]

    # Processors
    disparity_estimator: HitnetModel  # TODO: Add disparity estimator interface
    image_filter: Optional = None
    disparity_filter: Optional = None


def prepare_stereo_batch(
    stereo_camera: StereoCameraGroup, hitnet_path: Path
) -> StereoGeometryBatch:
    """Prepares a stereo geometry batch by setting up the camera data and processors."""

    disparity_estimator: HitnetModel = load_hitnet(hitnet_path).unwrap()

    logger.info(f"Camera pairs: {len(stereo_camera.camera_pairs)}")
    logger.info(f"Image loaders: {len(stereo_camera.image_loaders)}")

    if False:
        # TODO: Tune parameters
        image_filter = create_clahe_filter(clip=10.0, size=40)
    else:
        image_filter = None

    stereo_batch: StereoGeometryBatch = StereoGeometryBatch(
        # Data
        calibrations=stereo_camera.calibrations,
        camera_pairs=stereo_camera.camera_pairs,
        image_loaders=stereo_camera.image_loaders,
        # Processors
        disparity_estimator=disparity_estimator,
        image_filter=image_filter,
        disparity_filter=None,
    )

    return stereo_batch


def compute_stereo_geometry_batch(batch: StereoGeometryBatch) -> None:
    """Rectifies a stereo camera calibration, loads images, and computes range and normal maps.
    Exports the range and normal maps to image files."""

    windows: StereoWindows = create_stereo_windows()

    rectification: StereoRectificationResult = compute_stereo_rectification(
        batch.calibrations
    )

    for camera_pair in tqdm.tqdm(
        batch.camera_pairs, desc="Estimating stereo geometry..."
    ):

        loaders: Pair[ImageLoader] = Pair(
            batch.image_loaders.get(camera_pair.first),
            batch.image_loaders.get(camera_pair.second),
        )

        assert loaders.first is not None, "invalid first image loader"
        assert loaders.second is not None, "invalid second image loader"

        images: Pair[Image] = Pair(
            first=loaders.first(),
            second=loaders.second(),
        )

        assert images.first is not None, "invalid first image"
        assert images.second is not None, "invalid second image"

        geometry: StereoGeometry = compute_stereo_geometry(
            rectification=rectification,
            images=images,
            matcher=batch.disparity_estimator,
            image_filter=batch.image_filter,
            disparity_filter=batch.disparity_filter,
        )

        render_stereo_geometry(windows, geometry)

        key: KeyCode = wait_key_input(0)

        match key:
            case KeyCode.ESC:
                logger.info("Quitting...")
                destroy_all_windows()
                return
            case KeyCode.SPACE:
                continue
            case _:
                continue


def invoke_stereo_batch(
    target: GroupID,
    stereo_groups: list[StereoCameraGroup],
    hitnet_path: Path,
    range_directory: Path,
    normal_directory: Path,
) -> None:
    """Iterate over the stereo groups in a camera group."""

    stereo_group: StereoCameraGroup = stereo_groups[0]

    # TODO: Prepare export data
    # prepare_export_data(target, range_directory, normal_directory)

    # Prepare stereo estimation data
    stereo_batch: StereoGeometryBatch = prepare_stereo_batch(
        stereo_group, hitnet_path
    )

    # Perform stereo estimation - rectification, disparity, ranges, normals
    compute_stereo_geometry_batch(stereo_batch)


def main():
    """Main function."""

    # r29mrd5h_20090612_225306, r29mrd5h_20090613_100254, r29mrd5h_20110612_033752, r29mrd5h_20130611_002419
    # qdch0ftq_20100428_020202, qdch0ftq_20110415_020103, qdch0ftq_20120430_002423
    # r23685bc_20100605_021022, r23685bc_20120530_233021, r23685bc_20140616_225022
    # r23m7ms0_20100606_001908, r23m7ms0_20120601_070118, r23m7ms0_20140616_044549

    DOCUMENT_PATH: Path = Path(
        "/data/kingston_snv_01/acfr_metashape_projects/r23m7ms0_aligned_with_metadata.psz"
    )

    RANGE_DIRECTORY: Path = Path("/data/kingston_snv_01/stereo_export/r23m7ms0_20100606_001908_stereo_geometry")
    NORMAL_DIRECTORY: Path = Path("/data/kingston_snv_01/stereo_test/normals")

    environment: Environment = load_environment().unwrap()
    HITNET_PATH: Path = environment.resource_directory / Path(
        "hitnet_models/hitnet_eth3d_480x640.onnx"
    )

    DEPLOYMENT: str = "r23m7ms0_20140616_044549"

    match backend.load_project(DOCUMENT_PATH):
        case Ok(None):
            pass
        case Err(message):
            logger.error(message)

    groups: dict[str, GroupID] = {
        group.label: group for group in backend.get_group_identifiers().unwrap()
    }

    target: GroupID = groups.get(DEPLOYMENT)

    logger.info(f"Target deployment: {target}")

    match backend.camera_services.retrieve_stereo_cameras(target):
        case Ok(stereo_camera_groups):
            invoke_stereo_batch(
                target,
                stereo_camera_groups,
                hitnet_path=HITNET_PATH,
                range_directory=RANGE_DIRECTORY,
                normal_directory=NORMAL_DIRECTORY,
            )
        case Err(message):
            logger.error(message)


# ---------- Invoke main function ----------
main()